<h1><center> Twitter Sentiment Analysis

<h2> Introduction
     

<h2> Table of Content

<h2> Import Libraries

In [35]:
import pandas as pd
import sklearn
import numpy as np
import math
import matplotlib as plt
import seaborn as sns
import nltk
import re
import string
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

 <h2>Exploratory data analysis

<h3> Read Dataset

In [36]:
col_names = ['ID','Text','Selected_Text','Sentiment']
data = pd.read_csv('https://media.githubusercontent.com/media/dczzzzzdc/CSC590_Design_Project/main/Data/train.csv')
data.columns = col_names

In [37]:
sentiment_conv = {'negative':-1,'positive':1,'neutral':0}
data.drop(['Selected_Text','ID'],axis = 1,inplace = True)
data['Sentiment'] = data['Sentiment'].map(sentiment_conv)

<h3> Split Dataset

In [38]:
row_count = len(data.index)
train_row_count = round(row_count*0.8)
val_row_count =  row_count - train_row_count
train = data.iloc[:train_row_count,:]
val = data.iloc[train_row_count:,:]

<h3> Process Dataset